In [1]:
from gtfs_realtime_utils import *
from utils import *
import pandas as pd
ROOT = Path("../")
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/bus-tracking')

Parse GTFSRT data into pandas then clean it and write to CSV.

In [3]:
for date in range(20240915, 20240924):
    data = None
    gtfsrt_filepaths = None
    entities = None

    date = str(date)
    REALTIME_DATADIR = ROOT / f"data/gtfs-rt"
    gtfsrt_filepaths = gtfsrt_filepaths_to_list(dir=REALTIME_DATADIR, date=date)
    
    # Initialise the feed object
    feed = gtfs_realtime_pb2.FeedMessage()
    # Add all the entities (bus location objects) to a list to iterate through later.
    entities = entities_to_list(feed, gtfsrt_filepaths)
    print(f"Loaded data for {date}")
    # Add data to dataframe and sort
    data = entity_list_to_df(entities, date)

    print(f"Created dataframe for {date}")

    data.to_csv(ROOT / f"data/gtfs-rt/csv/{date}.csv", index=False)
    print(f'Finished writing {date}')

Loaded data for 20240915
28584494
Fraction of data for 20240915 that was duplictaed in 'longitude', 'latitude', 'timestamp', 'vehicle_id', 'trip_id':0.8568668383634848
Created dataframe for 20240915
Finished writing 20240915
Loaded data for 20240916
25628104
Fraction of data for 20240916 that was duplictaed in 'longitude', 'latitude', 'timestamp', 'vehicle_id', 'trip_id':0.6518878649782286
Created dataframe for 20240916
Finished writing 20240916
Loaded data for 20240917
28830261
Fraction of data for 20240917 that was duplictaed in 'longitude', 'latitude', 'timestamp', 'vehicle_id', 'trip_id':0.62329643841934
Created dataframe for 20240917
Finished writing 20240917
Loaded data for 20240918
28835781
Fraction of data for 20240918 that was duplictaed in 'longitude', 'latitude', 'timestamp', 'vehicle_id', 'trip_id':0.6373314806351178
Created dataframe for 20240918
Finished writing 20240918
Loaded data for 20240919
29227725
Fraction of data for 20240919 that was duplictaed in 'longitude', 'l